# Monitor

In this step we will create Monitor workflow pipeline based on schedule, whose purpose is to call

1) call the service created after training a data prediction model using darts framework and NBEATS Deep Learning model. (see notebook parcheggi_ml.ipynb)
2) save the prediction in database.


## Platform Support - Data Ops
We use the platform support to read the data created into the platform after the execution of notebook(parcheggi_data_pipeline.ipynb) for training

In [63]:
import pandas as pd
import requests
import os
import json
import digitalhub as dh
import getpass as gt

In [64]:
PROJECT_NAME = "parcheggi-nk-scheduler-"+gt.getuser()
proj = dh.get_or_create_project(PROJECT_NAME)
print("created project {}".format(PROJECT_NAME))
PROJECT_NAME

created project parcheggi-nk-scheduler-tenant2


'parcheggi-nk-scheduler-tenant2'

In [65]:
serving_fn = proj.get_function('serve_model')

In [66]:
data_item_download = proj.get_dataitem("dataset")

In [67]:
parkings_df = data_item_download.as_df()

In [68]:
parkings_df

,parcheggio,data,posti_liberi,posti_occupati,posti_totali,lat,lon
0,VIII Agosto,2024-06-07T01:59:00+00:00,484.0,141.0,625,44.500297,11.345368
1,Riva Reno,2024-06-07T02:09:00+00:00,369.0,101.0,470,44.501153,11.336062
2,Riva Reno,2024-06-07T02:19:00+00:00,369.0,101.0,470,44.501153,11.336062
3,VIII Agosto,2024-06-07T02:29:00+00:00,487.0,138.0,625,44.500297,11.345368
4,Riva Reno,2024-06-07T02:29:00+00:00,369.0,101.0,470,44.501153,11.336062
...,...,...,...,...,...,...,...
46670,VIII Agosto,2024-10-01T15:29:00+00:00,143.0,482.0,625,44.500297,11.345368
46671,Riva Reno,2024-10-01T15:29:00+00:00,263.0,207.0,470,44.501153,11.336062
46672,VIII Agosto,2024-10-01T15:39:00+00:00,149.0,476.0,625,44.500297,11.345368
46673,Autostazione,2024-10-01T15:39:00+00:00,127.0,138.0,265,44.504422,11.346514


In [69]:
pd. __version__

'1.4.4'

In [70]:
parkings_df = data_item_download.as_df()
parkings_df

,parcheggio,data,posti_liberi,posti_occupati,posti_totali,lat,lon
0,VIII Agosto,2024-06-07T01:59:00+00:00,484.0,141.0,625,44.500297,11.345368
1,Riva Reno,2024-06-07T02:09:00+00:00,369.0,101.0,470,44.501153,11.336062
2,Riva Reno,2024-06-07T02:19:00+00:00,369.0,101.0,470,44.501153,11.336062
3,VIII Agosto,2024-06-07T02:29:00+00:00,487.0,138.0,625,44.500297,11.345368
4,Riva Reno,2024-06-07T02:29:00+00:00,369.0,101.0,470,44.501153,11.336062
...,...,...,...,...,...,...,...
46670,VIII Agosto,2024-10-01T15:29:00+00:00,143.0,482.0,625,44.500297,11.345368
46671,Riva Reno,2024-10-01T15:29:00+00:00,263.0,207.0,470,44.501153,11.336062
46672,VIII Agosto,2024-10-01T15:39:00+00:00,149.0,476.0,625,44.500297,11.345368
46673,Autostazione,2024-10-01T15:39:00+00:00,127.0,138.0,265,44.504422,11.346514


In [71]:
import datetime 
import requests
import json
import pandas as pd
import digitalhub as dh

# Define a custom function to serialize datetime objects 
def serialize_datetime(obj): 
    if isinstance(obj, datetime.datetime): 
        return obj.isoformat() 
    raise TypeError("Type not serializable") 

# get current date and time as string
date_str = datetime.datetime.now().strftime('%Y-%m-%dT%H:%M:%S')
parkings_df = data_item_download.as_df()
# initialize an empty dataframe for predictions
pred_df = pd.DataFrame(columns=['parcheggio', 'datetime', 'predicted_mean'])
# get serving predictor function run
run_serve_model =  dh.get_run("store://parcheggi-nk-scheduler-tenant2/run/python+run/22153426-a270-4727-b69c-048bd6d94889", "parcheggi-nk-scheduler-tenant2")
# iterate over each parking in the dataset
# for index, row in parkings_df.iterrows():
    # parking_str = row['parcheggio']
#parking_str =  parkings_df['parcheggio'].iloc[0]
parcheggi =  parkings_df['parcheggio'].unique()
print(parcheggi)
# parking_str = parcheggi[0] 
# print(parking_str)
for parking_str in parcheggi:
    # construct API URL based on parking and current date
    API_URL = f'https://opendata.comune.bologna.it/api/explore/v2.1/catalog/datasets/disponibilita-parcheggi-storico/records?where=data%3C%3D%27{date_str}%27%20and%20parcheggio%3D%27{parking_str}%27&order_by=data%20DESC&limit=100'
    # define the file to store the latest data
    latest_data_file = 'last_records.json'
    # fetch data from the API and save it to a file
    with requests.get(API_URL) as r:
        with open(latest_data_file, "wb") as f:
            f.write(r.content)
        # read the latest data from the file and process it
        with open(latest_data_file) as f:
            json_data = json.load(f)
            df_latest = pd.json_normalize(json_data['results']).drop(columns=['guid', 'occupazione']).rename(columns={"coordinate.lon": "lon", "coordinate.lat": "lat"})
            df_latest.data = df_latest.data.astype('datetime64')
            df_latest['value'] = df_latest.posti_occupati / df_latest.posti_totali
            df_latest['date'] = df_latest.data.dt.round('30min')
            df_latest = df_latest.drop(columns=['parcheggio'])
            df_latest = df_latest.groupby('date').agg({'value': 'mean'})

        # convert the processed data to JSON and make a request to the serving predictor function
        jsonstr = df_latest.reset_index().to_json(orient='records')
        arr = json.loads(jsonstr)
        #print(arr)
        SERVICE_URL = run_serve_model.status.to_dict()["service"]["url"]
        with requests.post(f'http://{SERVICE_URL}', json={"inference_input":arr}) as r:
            res = json.loads(r.content)
            print(res)
        res_df = pd.DataFrame(res)
        res_df['datetime'] = res_df['date']
        res_df['parcheggio'] = parking_str
        res_df['predicted_mean'] = res_df['value']
        res_df = res_df.drop(columns=['date', 'value'])
        pred_df = pd.concat([pred_df, res_df], ignore_index=True)
        print(res_df['predicted_mean'])

      

['VIII Agosto' 'Riva Reno' 'Autostazione']
[{'date': 1727794800000, 'value': 0.8184624913}, {'date': 1727796600000, 'value': 0.8060316469}, {'date': 1727798400000, 'value': 0.7915043017}, {'date': 1727800200000, 'value': 0.7859667438}, {'date': 1727802000000, 'value': 0.8114300248}, {'date': 1727803800000, 'value': 0.7707617018}, {'date': 1727805600000, 'value': 0.7871003964}, {'date': 1727807400000, 'value': 0.7595436103}, {'date': 1727809200000, 'value': 0.8114669276}, {'date': 1727811000000, 'value': 0.8314693966}, {'date': 1727812800000, 'value': 0.8735354551}, {'date': 1727814600000, 'value': 0.8624750373}, {'date': 1727816400000, 'value': 0.8512861392}, {'date': 1727818200000, 'value': 0.7830797341}, {'date': 1727820000000, 'value': 0.7080538499}, {'date': 1727821800000, 'value': 0.5869241436}, {'date': 1727823600000, 'value': 0.4797517255}, {'date': 1727825400000, 'value': 0.3731933559}, {'date': 1727827200000, 'value': 0.2941035772}, {'date': 1727829000000, 'value': 0.219203892

In [90]:
# parcheggi =  parkings_df['parcheggio'].unique()
# for parking_str in parcheggi:

print(parkings_df.iterrows())

<generator object DataFrame.iterrows at 0x70691ce48430>


In [87]:
%%writefile "src/monitor_parking_predictions.py"
from digitalhub_runtime_python import handler
import datetime 
import requests
import json
import pandas as pd
import digitalhub as dh

@handler(outputs=["parking_data_predicted_model"])
def monitor_predictor(project,  parkings_di):
    """
    Monitor and predict parking occupancy.
    """

    # get serving predictor function run
    run_serve_model =  dh.get_run("store://parcheggi-nk-scheduler-tenant2/run/python+run/22153426-a270-4727-b69c-048bd6d94889", "parcheggi-nk-scheduler-tenant2")
    
    # get current date and time as string
    date_str = datetime.datetime.now().strftime('%Y-%m-%dT%H:%M:%S')

    # get parkings dataset and convert it to a dataframe
    parkings_df = parkings_di.as_df()

    # initialize an empty dataframe for predictions
    pred_df = pd.DataFrame(columns=['parcheggio', 'datetime', 'predicted_mean'])

    # iterate over each parking in the dataset
    for index, row in parkings_df.iterrows():
        parking_str = row['parcheggio']
        # construct API URL based on parking and current date
        API_URL = f'https://opendata.comune.bologna.it/api/explore/v2.1/catalog/datasets/disponibilita-parcheggi-storico/records?where=data%3C%3D%27{date_str}%27%20and%20parcheggio%3D%27{parking_str}%27&order_by=data%20DESC&limit=100'

        # define the file to store the latest data
        latest_data_file = 'last_records.json'

        # fetch data from the API and save it to a file
        with requests.get(API_URL) as r:
            with open(latest_data_file, "wb") as f:
                f.write(r.content)

        # read the latest data from the file and process it
        with open(latest_data_file) as f:
            json_data = json.load(f)
            df_latest = pd.json_normalize(json_data['results']).drop(columns=['guid', 'occupazione']).rename(columns={"coordinate.lon": "lon", "coordinate.lat": "lat"})
            df_latest.data = df_latest.data.astype('datetime64')
            df_latest['value'] = df_latest.posti_occupati / df_latest.posti_totali
            df_latest['date'] = df_latest.data.dt.round('30min')
            df_latest = df_latest.drop(columns=['parcheggio'])
            df_latest = df_latest.groupby('date').agg({'value': 'mean'})

        # convert the processed data to JSON and make a request to the serving predictor function
        jsonstr = df_latest.reset_index().to_json(orient='records')
        arr = json.loads(jsonstr)
        SERVICE_URL = run_serve_model.status.to_dict()["service"]["url"]
        with requests.post(f'http://{SERVICE_URL}', json={"inference_input":arr}) as r:
            res = json.loads(r.content)
        res_df = pd.DataFrame(res)
        res_df['datetime'] = res_df['date']
        res_df['parcheggio'] = parking_str
        res_df['predicted_mean'] = res_df['value']
        res_df = res_df.drop(columns=['date', 'value'])
        pred_df = pd.concat([pred_df, res_df], ignore_index=True)

    # concatenate the predicted results with the existing data (if any) and remove duplicates
    old_pd = pred_df
    try: 
        dat_old = project.get_dataitem('parking_prediction_nbeats_model')
        old_pd = pd.concat([dat_old.as_df(), pred_df], ignore_index=True)
        old_pd = old_pd.drop_duplicates(subset=['parcheggio', 'datetime'])
    except: pass

    # log the predictions as a dataset in the project
    project.log_dataitem('parking_prediction_nbeats_model', old_pd, stats=True)

    old_pd = pred_df.copy()
    old_pd['slice_datetime'] = date_str
    try:
        dat_old = project.get_dataitem('parking_prediction_nbeats_model_sliced')
        old_pd = pd.concat([dat_old.as_df(), old_pd], ignore_index=True)
    except: pass

    # log the predictions as a dataset in the project
    project.log_dataitem('parking_prediction_nbeats_model_sliced', old_pd, stats=True)

Writing src/monitor_parking_predictions.py


In [88]:
func = proj.new_function(name="monitor-parking-prediction",
                         kind="python",
                         python_version="PYTHON3_9",
                         source={"source": "src/monitor_parking_predictions.py", "handler": "monitor_predictor"})

In [89]:
data_item_download = proj.get_dataitem("dataset").key
run_monitor_parkings = func.run(action="job",inputs={"parkings_di": data_item_download},outputs={})